In [1]:
# devtools::install_github("NPSDC/beaveR", ref = "dev")
library(beaveR)
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tseCons <- buildTSE(treeTermFile = clustFile, coldata = coldata)

In [2]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file.path(saveDir, "cutsCons.RData"))
load(file.path(saveDir, "tseConsUp.RData"))
# save(tse, file=file.path(saveDir, "tseCons.RData"))

In [9]:
names(cuts)
nn <- c("lfcTPMDescNoMIRV", 'irv_height_1','irv_height_5','irv_height_10')
reqCuts <- cuts[nn]
load("environment/brain_sim_nodtu_lowfc/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tseCons)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tseCons)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tseCons)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])

temp <- tseCons
length(dts)
summary(abs(logFCNodes[dts]))
print(mean(abs(logFCNodes[dts]) >= rootFC))
rootFC
mcols(tseCons)[["meanInfRV"]][l+1]

[1] "irv_height_0.05"  "irv_height_0.1"   "irv_height_1"     "irv_height_5"    
 [5] "irv_height_10"    "lfcTPMDesc"       "LFC"              "lfcTPM"          
 [9] "lfcTPMDescNoMIRV" "lfcTPMNoMIRV"

[1] 8798

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4257  0.7799  1.0270  1.0093  1.2605  1.4836 

[1] 1


Node138201 
0.01270474

Node138201 
      0.01

In [7]:
names(reqCuts)

[1] "lfcTPMDescNoMIRV" "irv_height_1"     "irv_height_5"     "irv_height_10"

In [9]:
set.seed(1)
y <- fishpond::scaleInfReps(temp[1:l,])
y <- labelKeep(y)
y <- swish(y, x="condition")
detNodes <- list()
detNodes[["Txps"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))

negNodes <- list()
negNodes[["Txps"]] <- lapply(detNodes[["Txps"]], function(nodes) setdiff(seq(l), 
                                                          unlist(Descendants(tree, nodes))))
gse <- summarizeToGene(tseCons[1:l,])
dg <- union(dge.genes, dte.genes)
length(dg)
sum(dg %in% rownames(gse))
dg <- intersect(dge.genes, rownames(gse))

gse <- fishpond::scaleInfReps(gse)
gse <- labelKeep(gse)
set.seed(1)
gse <- swish(gse, x="condition")
detNodes[["Genes"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(gse)[,"qvalue"] <= x ))
negNodes[["Genes"]] <- lapply(detNodes[["Genes"]], function(nodes) setdiff(seq(nrow(gse)), nodes))

loading existing TxDb created: 2023-03-02 17:08:07

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2023-03-02 17:08:18

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



[1] 4150

[1] 4108

In [10]:
for(cuName in nn) {
    y <- temp[sort(cuts[[cuName]]),]
    y <- fishpond::scaleInfReps(y)
    y <- labelKeep(y)
    set.seed(1)
    y <- swish(y, x="condition")
    detNodes[[cuName]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
         match(nodes, names(logFCNodes))

    })
    negNodes[[cuName]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

In [10]:
detNodes <- list()
negNodes <- list()
nn <- c('irv_height_0.05', 'irv_height_0.1')
temp <- tseCons
for(cuName in nn) {
    y <- temp[sort(cuts[[cuName]]),]
    y <- fishpond::scaleInfReps(y)
    y <- labelKeep(y)
    set.seed(1)
    y <- swish(y, x="condition")
    detNodes[[cuName]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
         match(nodes, names(logFCNodes))

    })
    negNodes[[cuName]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

In [11]:
1

[1] 1

In [11]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    if(!n%in%c("term", "Genes")) {
        print(n)
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        df <- rbind(df, stats)
        
    }
    
}
print(df)

[1] "Txps"
[1] "tp 0.990276157137301"
[1] "tp 0.962688916876574"
[1] "tp 0.92637778712663"
[1] "lfcTPMDescNoMIRV"
[1] "tp 0.999080882352941"
[1] "tp 0.962388059701493"
[1] "tp 0.919754732071448"
[1] "irv_height_1"
[1] "tp 0.998256624825662"
[1] "tp 0.964522058823529"
[1] "tp 0.9230644375913"
[1] "irv_height_5"
[1] "tp 0.997685950413223"
[1] "tp 0.963011031797534"
[1] "tp 0.920444824636441"
[1] "irv_height_10"
[1] "tp 0.990300678952473"
[1] "tp 0.956148564294632"
[1] "tp 0.919431279620853"
               Method Metric     FDR_0.01   FDR_0.05   FDR_0.10
fdr              Txps    FDR  0.009723843 0.03731108 0.07362221
tpr              Txps    TPR    0.5787679  0.6950443  0.7508525
fdr1 lfcTPMDescNoMIRV    FDR 0.0009191176 0.03761194 0.08024527
tpr1 lfcTPMDescNoMIRV    TPR    0.5014994   0.743714  0.7958478
fdr2     irv_height_1    FDR  0.001743375 0.03547794 0.07693556
tpr2     irv_height_1    TPR    0.3916017  0.7176857   0.777869
fdr3     irv_height_5    FDR   0.00231405 0.03698897 0.079

In [12]:
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfA)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfA)-1))
    dfA[,i+1] <- round(as.numeric(unlist(dfA[,i+1])),3)
dfA

Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Txps,0.010,0.037,0.074,0.579,0.695,0.751
lfcTPMDescNoMIRV,0.001,0.038,0.080,0.501,0.744,0.796
irv_height_1,0.002,0.035,0.077,0.392,0.718,0.778
irv_height_5,0.002,0.037,0.080,0.360,0.709,0.771
irv_height_10,0.010,0.044,0.081,0.593,0.712,0.766


In [12]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    if(!n%in%c("term", "Genes")) {
        print(n)
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        df <- rbind(df, stats)
        
    }
    
}
print(df)
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfA)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfA)-1))
    dfA[,i+1] <- round(as.numeric(unlist(dfA[,i+1])),3)
dfA

[1] "irv_height_0.05"
[1] "tp 0.989348370927318"
[1] "tp 0.958007303077726"
[1] "tp 0.915744881148505"
[1] "irv_height_0.1"
[1] "tp 0.989531840651352"
[1] "tp 0.962891098714528"
[1] "tp 0.925258298527149"
              Method Metric   FDR_0.01  FDR_0.05   FDR_0.10
fdr  irv_height_0.05    FDR 0.01065163 0.0419927 0.08425512
tpr  irv_height_0.05    TPR  0.6288331 0.7313819  0.7747909
fdr1  irv_height_0.1    FDR 0.01046816 0.0371089  0.0747417
tpr1  irv_height_0.1    TPR  0.6239457 0.7279061  0.7717272


Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
irv_height_0.05,0.011,0.042,0.084,0.629,0.731,0.775
irv_height_0.1,0.010,0.037,0.075,0.624,0.728,0.772
